# Feature Selection & Dimensionality Reduction
Try out different feature combinations, and PCA/LDA.

In [10]:
feature_selection_method = "none"
# Possible: "manual", "none"

dim_reduction_method = "pca"
# Possible: "pca", "lda", "none"

In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

### Load Data

In [12]:
train_preprocessed_df = pd.read_csv("final_df.csv",  na_values = [0, '0'])
train_preprocessed_df = train_preprocessed_df.drop('Unnamed: 0', axis=1)
labels = pd.read_csv("project/data/train/labels.csv")

#test_df = pd.read_csv("project/data/test/test.csv", parse_dates = ['date_recorded' ],  na_values = [0, '0'])

# Merge features and labels files
train_df = pd.merge(labels, train_preprocessed_df, on='id')

# Numerical cols
numerical_cols = train_df.select_dtypes(include='number').columns.tolist()
categorical_cols = ['funder',
 'installer',
 'basin',
 'region_code',
 'extraction_type',
 'extraction_type_class',
 'payment_type',
 'water_quality',
 'quantity',
 'source',
 'waterpoint_type',
 'scheme_management',
 'season',
 'region_district',
 'authority_scheme',
 'population_bins']
non_numerical_cols = train_df.select_dtypes(exclude='number').columns.tolist()

print(f"Numerical Cols: {numerical_cols}")
print(f"Encoded Categorical Cols: {non_numerical_cols}")

Numerical Cols: ['id', 'Unnamed: 0', 'funder', 'installer', 'longitude', 'latitude', 'basin', 'region_code', 'public_meeting', 'extraction_type', 'extraction_type_class', 'payment_type', 'water_quality', 'quantity', 'source', 'waterpoint_type', 'gps_height', 'permit', 'scheme_management', 'population', 'recorded_age', 'amount_tsh_missing', 'season', 'region_district', 'authority_scheme']
Non Numerical Cols: ['status_group_x', 'status_group_y']


## Feature Selection

### No Feature Selection
Simply use all of the processed/engineered features.

In [13]:
if(feature_selection_method == "none"):
    pass

### Manual Feature Selection
BrendaLoznik drops these features:

`'amount_tsh', 'date_recorded',  'extraction_type_group', 'lga', 'management', 'management_group', 'num_private', 'payment', 'source_class', 'source_type', 'quality_group', 'quantity_group', 'ward', 'waterpoint_type_group', 'wpt_name', 'scheme_name', 'subvillage', 'recorded_by', 'id', 'status_group'`

In [14]:
if(feature_selection_method == "manual"):
    features_to_drop = ['amount_tsh', 'date_recorded',  'extraction_type_group', 'lga', 'management', 'management_group',
                        'num_private', 'payment', 'source_class', 'source_type', 'quality_group', 'quantity_group', 'ward',
                        'waterpoint_type_group', 'wpt_name', 'scheme_name', 'subvillage', 'recorded_by', 'id', 'status_group']
    train_df = train_df.drop(features_to_drop)

## Feature Normalisation

### Min-Max Scaling


### Z-score Normalisation

### Combined Scaling
Manually combine Min-Max scaling, Z-score normalisation and Robust scaling on a feature-by-feature basis.

## Dimensionality Reduction

### PCA

In [15]:
if(dim_reduction_method=="pca"):
    

IndentationError: expected an indented block (2393878603.py, line 2)